In [1]:
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np
from datetime import datetime
import math

In [29]:
class Psi:
    def __init__(self, path=r'C:\Users\JGreenw9\JNJ\External Operations Plan - PSI Reports'):
        self.path = path
        self.files = self.get_files(self.path)
        self.filepaths = self.get_filepaths(self.path, self.files)

    def get_files(self, path):
        return [f for f in listdir(path) if isfile(join(path, f)) and '.xl' in f]

    def get_filepaths(self, path, files):
        return [path + '\\'+ x for x in files]

    def read_file(self, filepath):
        df = pd.DataFrame()
        if '.xlsx' in filepath[-5:]:
            df = pd.read_excel(filepath)
        elif '.xlsb' in filepath[-5:]:
            df = pd.read_excel(filepath, engine='pyxlsb')
        else:
            pd.DataFrame()
        return df

    def get_date_header_conv(self, df):
        excel_dates = []
        date_time : list[datetime] = []
        for x in df.columns:
            try:
                int(x)
                excel_dates.append(x)
            except (ValueError, TypeError) as e:
                if type(x) == datetime:
                    date_time.append(x) 
                else:
                    pass               
        dates = {}
        for date in excel_dates:
            dt = datetime.fromordinal(datetime(1900, 1, 1).toordinal() + int(date) - 2)
            result = str(dt.date())
            dates[date] = result
        for date in date_time:
            result = str(date.date())
            dates[date] = result
        return dates

    def init_prep_df(self, df : pd.DataFrame):
        df.columns = df.iloc[0]
        df = df.iloc[1:]
        renames = self.get_date_header_conv(df)
        df = df.rename(columns=renames)
        return df

    def get_oh(self, df):
        df = df[(df['Key Figure'] == 'Total Demand') | (df['Key Figure'] == 'Projected Inventory')]
        df = df[['LocationId', 'ProductId', '2014-01-01']].groupby(by=['LocationId', 'ProductId']).sum().reset_index()
        df['Key Figure'] = 'On Hand Inventory'
        df['Date'] = '2014-01-01'
        df = df.rename(columns={'2014-01-01': 'value'})
        return df

    def get_supply(self, df):
        df = df
        df = df[df['VendorDesc'].str.contains('-')]
        df = df[df['Key Figure'].isin(['Firm PO Receipt', 'Planned Order Receipt', 'Stock Movement'])]
        dates = [x for x in df.columns if '20' in x]
        df = pd.melt(df, ['LocationId', 'ProductId', 'Key Figure'], dates, var_name='Date')
        df = df[df['value'] != 0].reset_index(drop=True)
        return df

    def date_offset(self, x, dates):
        row = dates.index(x['Date'])
        new_row = row + math.ceil(x['GRPT (Wks)'])
        result = np.nan
        try:
            result = dates[new_row]
        except:
            pass
        return result

    def get_inventory(self, df : pd.DataFrame):
        oh_inventory = self.get_oh_all(df)
        df = df[df['Key Figure'].isin(['Firm PO Receipt'])]
        dates = [x for x in df.columns if '20' in x]
        not_dates = [x for x in df.columns if '20' not in x]
        df = pd.melt(df, not_dates, dates, var_name='Date')
        df['value'] = df['value'].astype(np.float64)
        df['Key Figure'] = 'Inventory'
        first_date = df['Date'] == '2014-01-01'
        stock_in_routing = df[first_date].copy(deep=True)
        pos_by_arrival = df[~first_date].copy(deep=True)
        stock_in_routing['value'] = stock_in_routing['StockInRouting']
        stock_in_routing['Date'] = dates[1]
        date_offset = self.date_offset
        pos_by_arrival['Date'] = pos_by_arrival.apply(lambda x: date_offset(x, dates), axis=1)
        pos_by_arrival = pos_by_arrival[~pos_by_arrival['Date'].isna()]
        pos_by_arrival = pos_by_arrival[pos_by_arrival['value'] != 0]
        stock_in_routing = stock_in_routing[stock_in_routing['value'].astype('int64') != 0]
        df = pd.concat([oh_inventory, stock_in_routing, pos_by_arrival]).reset_index(drop=True)
        df['value'] = df['value'].astype(np.float64)
        cols = df.columns.to_list()
        not_cols = ['value', 'Date']
        pivot_cols = [x for x in cols if x not in not_cols]
        cal = pd.read_csv(r'\\na.jnj.com\dpyusdfsroot\RY_Company\Supply Chain Mgmt\Spine Plan-NPI\Teligen\Alteryx Deployment\Inputs\JNJ Calendar.csv')
        df = df.merge(cal[['Date (YYYY-MM-DD)', 'JNJ Week (YYYY/WW)', 'YYYYMM']],left_on=['Date'],right_on='Date (YYYY-MM-DD)').drop('Date (YYYY-MM-DD)', axis=1)
        df = pd.pivot_table(df, values='value', index=pivot_cols, columns=['Date'], aggfunc=np.sum).reset_index()
        return df

    def get_oh_all(self, df : pd.DataFrame):
        df = df[(df['Key Figure'] == 'Total Demand') | (df['Key Figure'] == 'Projected Inventory')]
        not_dates = [x for x in df.columns if '20' not in x]
        df = df[not_dates+['2014-01-01']].groupby(by=not_dates).sum().reset_index()
        df['Key Figure'] = 'Inventory'
        df['Date'] = '2014-01-01'
        df = df.rename(columns={'2014-01-01': 'value'})
        return df

    def combine_dfs(self, supply, onhand):
        df = pd.concat([onhand, supply]).reset_index(drop=True)
        df['Key Figure'] = df['Key Figure'].astype(str)
        df = pd.pivot(df, ['LocationId', 'ProductId', 'Date'], 'Key Figure', 'value').reset_index()
        cal = pd.read_csv(r'\\na.jnj.com\dpyusdfsroot\RY_Company\Supply Chain Mgmt\Spine Plan-NPI\Teligen\Alteryx Deployment\Inputs\JNJ Calendar.csv')
        df = df.merge(cal[['Date (YYYY-MM-DD)', 'YYYYMM']],left_on=['Date'],right_on='Date (YYYY-MM-DD)').drop('Date (YYYY-MM-DD)', axis=1)
        df = df.sort_values(['ProductId', 'Date', 'LocationId'])
        return df

    def final_df(self, df):
        supply = self.get_supply(df)
        onhand = self.get_oh(df)
        df = self.combine_dfs(supply, onhand)
        return df

    def all_files(self, filepaths):
        dfs = []
        raw_dfs = []
        for filepath in filepaths:
            print(filepath)
            df = self.read_file(filepath)
            raw_dfs.append(df)
            df = self.init_prep_df(df)
            df = self.final_df(df)
            dfs.append(df)
        df = pd.concat(dfs)
        return df, raw_dfs

In [3]:
psi = Psi()
psi.filepaths = [r'C:\Users\JGreenw9\Downloads\PSI 22-06-27 BW partial.xlsx']
data, raw_dfs = psi.all_files(psi.filepaths)

C:\Users\JGreenw9\Downloads\PSI 22-06-27 BW partial.xlsx


In [4]:
data.to_clipboard()

In [33]:
df = Psi().init_prep_df(raw_dfs[0])
inv = Psi().get_inventory(df)
df = pd.concat([df, inv])
df['Key Figure'] = df['Key Figure'].astype('category')
df['Key Figure'].cat.set_categories(df['Key Figure'].unique().tolist(), inplace=True)
df = df.sort_values(['ProductId', 'LocationId', 'Key Figure'])
df.to_clipboard()
df

c:\Users\JGreenw9\Desktop\Python\data-utility\venv\lib\site-packages\pandas\core\arrays\categorical.py:2747: FutureWarning: The `inplace` parameter in pandas.Categorical.set_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  res = method(*args, **kwargs)


,Profile,LocationId,VendorDesc,P4,ProductId,Description,MOQ,GRPT (Wks),Fixed Horizon,StockInRouting,...,2024-04-15,2024-04-22,2024-04-29,2024-05-06,2024-05-13,2024-05-20,2024-05-27,2024-06-03,2024-06-10,2024-06-17
1,ExOps Bridgewater 06/DS_SPN1/HUB,MORS,0430098878 - Medela AG,TELIGEN,101043438,TELIGEN PUMP CONVENIENCE KIT,1,0,10w 1d,104,...,1.5,1.5,1.5,1.5,1.5,1.5,1.2,1.2,1.2,1.2
2,ExOps Bridgewater 06/DS_SPN1/HUB,MORS,0430098878 - Medela AG,TELIGEN,101043438,TELIGEN PUMP CONVENIENCE KIT,1,0,10w 1d,104,...,0,0,0,0,0,0,0,0,0,0
3,ExOps Bridgewater 06/DS_SPN1/HUB,MORS,0430098878 - Medela AG,TELIGEN,101043438,TELIGEN PUMP CONVENIENCE KIT,1,0,10w 1d,104,...,0,0,0,0,0,0,0,0,0,0
4,ExOps Bridgewater 06/DS_SPN1/HUB,MORS,0430098878 - Medela AG,TELIGEN,101043438,TELIGEN PUMP CONVENIENCE KIT,1,0,10w 1d,104,...,0,6,0,0,0,6,0,0,0,0
5,ExOps Bridgewater 06/DS_SPN1/HUB,MORS,0430098878 - Medela AG,TELIGEN,101043438,TELIGEN PUMP CONVENIENCE KIT,1,0,10w 1d,104,...,0,0,6,0,0,0,6,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,ExOps Bridgewater 05/DS_SPF3/HUB,MORS,0430084709 - Tyber Medical LLC,PROTI 360 TPAL,108812214,"T-PAL PROTI, 12X14X32 MM",25,2,19w 0d,0,...,0,0,0,0,0,0,0,0,0,0
449,ExOps Bridgewater 05/DS_SPF3/HUB,MORS,0430084709 - Tyber Medical LLC,PROTI 360 TPAL,108812214,"T-PAL PROTI, 12X14X32 MM",25,2,19w 0d,0,...,0,0,0,0,0,0,0,0,0,0
450,ExOps Bridgewater 05/DS_SPF3/HUB,MORS,0430084709 - Tyber Medical LLC,PROTI 360 TPAL,108812214,"T-PAL PROTI, 12X14X32 MM",25,2,19w 0d,0,...,3,3,3,3,3,3,3,3,3,3
451,ExOps Bridgewater 05/DS_SPF3/HUB,MORS,0430084709 - Tyber Medical LLC,PROTI 360 TPAL,108812214,"T-PAL PROTI, 12X14X32 MM",25,2,19w 0d,0,...,28,28,28,28,28,28,28,28,28,28
